## Graph Coverage Analysis for Service Points

This notebook demonstrates how to calculate coverage zones from service points through a multimodal transportation network using Dijkstra's algorithm and Voronoi diagrams.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install iduedu pyarrow objectnat

In [ ]:
# Import dependencies
from iduedu import get_intermodal_graph, get_boundary
import geopandas as gpd
from objectnat import get_graph_coverage
import os

### 1. Load Transportation Network
First, we retrieve the multimodal graph (roads, public transport, etc.) for a specified region using its OSM ID.

In [ ]:
# Get city boundary and transportation network
poly = get_boundary(osm_id=1114252)  # Example OSM ID for a city
G_intermodal = get_intermodal_graph(polygon=poly, clip_by_bounds=True)

### 2. Load Service Points
These represent locations (e.g., healthcare facilities, schools) for which we want to calculate coverage zones.

In [ ]:
# Load service points (replace with your actual data path)
services = gpd.read_parquet('examples_data/services.parquet')

### 3. Calculate Coverage by Distance
Creates service areas based on maximum travel distance (800 meters in this example).

In [ ]:
# Calculate coverage zones by distance (800m cutoff)
result_length = get_graph_coverage(
    gdf_from=services,
    nx_graph=G_intermodal,
    weight_type="length_meter",
    weight_value_cutoff=800
)

# Visualize results
result_length.explore(column='name', tiles='CartoDB Positron')

### 4. Calculate Coverage by Travel Time
Creates service areas based on maximum travel time (10 minutes in this example), clipped to the city boundary.

In [ ]:
# Prepare zone boundary
zone = gpd.GeoDataFrame(geometry=[poly], crs=4326)

# Calculate coverage zones by time (10min cutoff)
result_time = get_graph_coverage(
    gdf_from=services,
    nx_graph=G_intermodal,
    weight_type="time_min",
    weight_value_cutoff=10,
    zone=zone
)
# Visualize results
result_time.explore(column='name', tiles='CartoDB Positron')

### Key Parameters Explained:
- `weight_type`: 
  - `"length_meter"` for distance-based coverage
  - `"time_min"` for time-based coverage
- `weight_value_cutoff`: Maximum travel distance/time threshold
- `zone` (optional): Boundary polygon to clip results